In [2]:
import numpy as np
import pandas as pd
import scipy.stats as st
import numpy as np
import math

## z-score и t-score

### Поиск z-score по площади и площади по z-score

In [6]:
# поиск площади слева от z-score, например при z-score = 1.96
print(st.norm.cdf(1.96)) # должно получиться 0.975
# если взять -1.96
print(st.norm.cdf(-1.96)) # должно получиться 0.025 потому что колокол симметричен

# если мы хотим найти площадь справа от 1.96 (правый хвост, например при тестированни правосторонней гипотезы)
print(1 - st.norm.cdf(1.96))

# площадь между -1.96 и 1.96, из большей площади вычитаем меньшую
print(st.norm.cdf(1.96) - st.norm.cdf(-1.96))

0.9750021048517795
0.024997895148220435
0.024997895148220484
0.950004209703559


In [11]:
# обратный процесс, поиск z-score по площади (например, 97.5% площади слева от z-score, которое мы ищем)
print(st.norm.ppf(0.975))

# показать что процессы взаимообратны можно так (погрешность связана с округлением)
print(st.norm.ppf(st.norm.cdf(1.96)))

1.959963984540054
1.959999999999999


### Поиск t-score по площади и площади по t-score

In [25]:
# поиск площади слева от t-score (первый параметр - t-score, второй - df, степени свободы, объем выборки - 1)
print(st.t.sf(3.078, 1))

# площади справа и между двумя t-score считаются аналогично z-score

# поиск t-score по площади
# допустим выборка n = 15, тогда степеней свободы 14
df = 14
print(st.t.ppf(0.95, df))

0.09999038172554342
2.1447866879169273


## Confidence intervals

Доверительный интервал для среднего значения (вариант 1). Нормальное распределение генеральной совокупности или выборка (n) > 30. **СКО генеральной совокупности известно**. Используем z-score.

In [20]:
# размер выборки и уровень доверия задаются исследователем
# СКО ген. совокупности для этого метода должно быть известно заранее

# confidence level
conf = 0.95

# размер выборки
n = 35

# стандартное отклонение ген. совокупности
stdev_population = 15

# считаем среднее по выборке и например получаем 7
mean = 7

# тогда интервал считается как среднее +- погрешность (Margin of error, ME, E)
# погрешность считается как E = z-score * stdev / sqrt(n)

# сначала найдем z-score
# так как мы рассматриваем площадь слева и справа от среднего, то для уровня доверия 0.95 нужно посчитать z-score
# либо для 0.025 и поменять знак, либо для 0.975 (удобнее второе) - ниже даны два одинаковых варианта
print(st.norm.ppf(0.975))
print(st.norm.ppf((1+0.95)/2))

1.959963984540054
1.959963984540054


In [26]:
# считаем погрешность
z = st.norm.ppf((1+0.95)/2)
print(z)
E = z * stdev_population / math.sqrt(n)
print(E) # это и есть наше отклонение от среднего при уровне доверия 95%

# строим интервал
print(mean - E, mean + E)

1.959963984540054
4.969415701946049
2.0305842980539506 11.96941570194605


In [27]:
# этот результат означает, что с 95% вероятностью можно утверждать, что истинное среднее ген. совокупности находится
# в этом интервале

# другая интерпретация: если взять 100 выборок, то можно ожидать, что 95 из них будут содержать истинное среднее

Доверительный интервал для среднего значения (вариант 2). Нормальное распределение генеральной совокупности или выборка (n) > 30. **СКО генеральной совокупности НЕизвестно**. Используем t-score.

In [31]:
conf = 0.95
n = 35
mean = 7
stdev_sample = 15 # СКО выборки, в отличие от СКО ген. совокупности известно всегда

df = n - 1 # степени свободы, здесь 35 - 1 = 34
t = st.t.ppf((1+conf)/2, df) # t-score для two-tailed test
print(t)

# погрешность
E = t * stdev_sample / math.sqrt(n)
print(E)

# интервал
print(mean - E, mean + E)

2.032244509317718
5.152680281095608
1.8473197189043917 12.15268028109561


In [ ]:
# интерпретация интервала аналогична
# обратите внимание, t-distribution дает гораздо более широкий интервал (более консертавтивная оценка)
# при аналогичных параметрах среднего, выборки и СКО (правда здесь для выборки, а не ген. совокупности)

#### Минимальный размер выборки

In [ ]:
# зная Margin of error (погрешность) и желаемый уровень доверия можно посчитать
# минимальный размер выборки для z-score и t-score (выводится из формулы погрешности (ME))

In [ ]:
# через z-score (известно СКО ген. совокупности)
conf = 0.95
z = st.norm.ppf((1 + conf)/2 # two-tailed z-score
n = ((z * stdev_population) / E) ** 2

# через t-score
# минимальный размер выборки посчитать нельзя, потому что для определения t-score уже нужно знать размер выборки
# (от этого зависят степени свободы)

## Hypothesis testing (one sample)

### Population standard deviation known. Z-test

Тестирование гипотезы для среднего значения генеральной совокупности. СКО ген. совокупности известно. Ген. совокупность либо нормально распределена, либо выборка > 30.

Условие примера: считается, что средний возраст шимпанзе 20 лет. Вы как исследователь с этим не согласны. Тогда:

- Нулевая гипотеза: средний возраст шимпанзе 20 лет
- Альтернативная гипотеза: средний возраст шимпанзе НЕ 20 лет (two-tailed test)

При этом известен СКО ген. совокупности (на практике этот параметр никогда не бывает известен, поэтому используется t-test).

In [17]:
# Вариант 1. Расчет вручную.

alpha = 0.05 # уровень значимости, пороговое значение, задается исследователем
conf = 1 - alpha # уровень доверия 0.95

population_mean = 20
population_sd = 11

n = 45
sample_mean = 21

z_critical = st.norm.ppf((1 + 0.95) / 2)

z_calc = population_mean - sample_mean / (population_sd / math.sqrt(n))
z_calc

p_value = 2 * st.norm.cdf(z_calc)
p_value

1.999999999999368

In [16]:
# Вариант 2. Решение из коробки


